# Entrant Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys

import json
import unittest

from common import Printable, projdir

from constants import *

## Constants

In [3]:
ID_KEY = 'ID'
NAME_KEY = 'Name'

GENDER_KEY = 'Gender'
NATIONALITY_KEY = 'Nationality'
COUNTRY_KEY = 'Country'

CRAFT_TYPE_KEY = 'Craft Type'
STATUS_KEY = 'Status'
NOVICE_KEY = 'Novice'
FIRST_TIMER_KEY = 'First Timer'
ENTRY_TYPE_KEY = 'Entry Type'

UKWA_MEMBER_KEY = 'UKWA Member'
ISWC_MEMBER_KEY = 'ISWC Member'

GT31_ID_KEY = 'GT31 ID'
GT31_SN_KEY = 'GT31 SN'

## Entrant Class

Class to manage entrants

In [4]:
class Entrant(Printable):
    def __init__(self, headers=[], values=[], verbosity=1):
        '''Initialise entrant object'''

        super().__init__(verbosity=verbosity)

        # Quickly check the number of values matches the header
        if len(headers) != len(values):
            raise ValueError('Unexpected number of fields for entrant ID {}'.format(values[0]))

        # Patch legacy Excel headers to use desired field names
        self.patchHeaders(headers)

        # Store entrant details in a dictionary - easy for applying report filters
        self.entrantDict = {}
        for i in range(len(values)):
            if headers[i] == ID_KEY:
                self.entrantDict[headers[i]] = int(values[i])
            else:
                self.entrantDict[headers[i]] = values[i]

        # Patch values in dictionary
        self.patchDictionary()

        # Validate the dictionary to ensure values are in the config
        self.validateDictionary()

        # Record GPS details
        self.parseGt31Details()

        # Note: A session is a complete day in WSW terms but may comprise of multiple courses
        self.sessions = {}


    def getValue(self, name):
        '''Retrieve value from dictionary'''

        return self.entrantDict[name]


    def patchHeaders(self, headers):
        '''Tweak headers for consistency across years'''

        for i in range(len(headers)):
            # All years (avoid the slash)
            if headers[i] == 'Male/Female':
                headers[i] = GENDER_KEY

            # 2012 onwards
            elif headers[i] == 'EntryType':
                headers[i] = ENTRY_TYPE_KEY

            # 2010 + 2011
            elif headers[i] == 'requiredEntryType':
                headers[i] = ENTRY_TYPE_KEY

            # 2010
            elif headers[i] == 'Novice(Y/N)':
                headers[i] = NOVICE_KEY

            # 2010
            elif headers[i] == 'NMA Member':
                headers[i] = ISWC_MEMBER_KEY


    def patchDictionary(self):
        '''Patch dictionary values to handle previous years'''

        self.patchName()
        self.patchGender()
        self.patchNationality()

        self.patchCraftType()
        self.patchNovice()
        self.patchFirstTimer()
        self.patchStatus()
        self.patchEntryType()

        self.patchUKWA()
        self.patchISWC()


    def patchName(self):
        '''Derive full name from First Name and Family Name'''

        if 'First Name' in self.entrantDict:
            firstName = self.entrantDict['First Name'].strip()
        else:
            firstName = None

        if 'Family Name' in self.entrantDict:
            familyName = self.entrantDict['Family Name'].strip()
        else:
            familyName = None

        if firstName and familyName:
            self.entrantDict[NAME_KEY] = firstName + ' ' + familyName
        elif firstName:
            self.entrantDict[NAME_KEY] = firstName
        elif familyName:
            self.entrantDict[NAME_KEY] = familyName
        elif GT31_ID_KEY in self.entrantDict and self.entrantDict[GT31_ID_KEY]:
            self.entrantDict[NAME_KEY] = self.entrantDict[GT31_ID_KEY]
        else:
            self.entrantDict[NAME_KEY] = 'TBC'


    def patchGender(self):
        '''Patch gender in dictionary to handle possible typos'''

        if GENDER_KEY in self.entrantDict:
            self.entrantDict[GENDER_KEY] = self.entrantDict[GENDER_KEY][:1].upper()

            if self.entrantDict[GENDER_KEY] not in config['Entrants'][GENDER_KEY]:
                self.entrantDict[GENDER_KEY] = '?'
        else:
            self.entrantDict[GENDER_KEY] = '?'


    def patchNationality(self):
        '''Patch nationality in dictionary to handle previous years'''

        # If nationality is not present then default to the country of residence
        if NATIONALITY_KEY not in self.entrantDict or self.entrantDict[NATIONALITY_KEY] == '':
            if COUNTRY_KEY in self.entrantDict:
                self.entrantDict[NATIONALITY_KEY] = self.entrantDict[COUNTRY_KEY]

        # It's not guaranteed that nationality is present yet
        if NATIONALITY_KEY in self.entrantDict:
            # Remove leading and trailing whitespace
            self.entrantDict[NATIONALITY_KEY] = self.entrantDict[NATIONALITY_KEY].strip()

            if self.entrantDict[NATIONALITY_KEY]:
                # Convert England to United Kingdom so that ISO code + flag can be used for results
                if self.entrantDict[NATIONALITY_KEY] in ('England', 'Scotland', 'Wales', 'Northern Ireland'):
                    self.entrantDict[NATIONALITY_KEY] = 'United Kingdom'
            else:
                self.entrantDict[NATIONALITY_KEY] = 'TBC'
        else:
            self.entrantDict[NATIONALITY_KEY] = 'TBC'


    def patchCraftType(self):
        '''Patch craft type in dictionary to handle unexpected values'''

        if CRAFT_TYPE_KEY in self.entrantDict:
            if self.entrantDict[CRAFT_TYPE_KEY] in ('Kite-Board', 'Kite Board'):
                self.entrantDict[CRAFT_TYPE_KEY] = 'Kiteboard'

            elif self.entrantDict[CRAFT_TYPE_KEY] in ('Wing-Board', 'Wing Board'):
                self.entrantDict[CRAFT_TYPE_KEY] = 'Wingboard'

            if self.entrantDict[CRAFT_TYPE_KEY] not in config['Entrants'][CRAFT_TYPE_KEY]:
                self.entrantDict[CRAFT_TYPE_KEY] = 'TBC'
        else:
            self.entrantDict[CRAFT_TYPE_KEY] = 'TBC'


    def patchNovice(self):
        '''Patch novice in dictionary to guarantee Y, N or ?'''

        if STATUS_KEY in self.entrantDict:
            # 2011
            if self.entrantDict[STATUS_KEY] == 'Novice':
                self.entrantDict[NOVICE_KEY] = 'Y'
                
            # Gold fleet and Pro fleet are exempt from being first timers
            elif self.entrantDict[STATUS_KEY]  in ['Gold-Fleet', 'Gold Fleet', 'Professional', 'Pro-Fleet', 'Pro Fleet']:
                self.entrantDict[NOVICE_KEY] = 'N'

        # Generic logic
        if NOVICE_KEY in self.entrantDict:
            self.entrantDict[NOVICE_KEY] = self.entrantDict[NOVICE_KEY][:1].upper()

            if self.entrantDict[NOVICE_KEY] not in config['Entrants'][NOVICE_KEY]:
                self.entrantDict[NOVICE_KEY] = '?'
        else:
            self.entrantDict[NOVICE_KEY] = '?'


    def patchFirstTimer(self):
        '''Patch first timer in dictionary to guarantee Y, N or ?'''

        # Gold fleet and Pro fleet are exempt from being novice
        if STATUS_KEY in self.entrantDict:
            if self.entrantDict[STATUS_KEY]  in ['Gold-Fleet', 'Gold Fleet', 'Professional', 'Pro-Fleet', 'Pro Fleet']:
                self.entrantDict[FIRST_TIMER_KEY] = 'N'

        # Generic logic
        if FIRST_TIMER_KEY in self.entrantDict:
            self.entrantDict[FIRST_TIMER_KEY] = self.entrantDict[FIRST_TIMER_KEY][:1].upper()

            if self.entrantDict[FIRST_TIMER_KEY] not in config['Entrants'][FIRST_TIMER_KEY]:
                self.entrantDict[FIRST_TIMER_KEY] = '?'
        else:
            self.entrantDict[FIRST_TIMER_KEY] = '?'



    def patchStatus(self):
        '''Patch status in dictionary to handle previous years'''

        if FIRST_TIMER_KEY in self.entrantDict and self.entrantDict[FIRST_TIMER_KEY] == 'Y':
            self.entrantDict[STATUS_KEY] = 'First Timer'

        elif NOVICE_KEY in self.entrantDict and self.entrantDict[NOVICE_KEY] == 'Y':
            self.entrantDict[STATUS_KEY] = 'Novice'

        if STATUS_KEY in self.entrantDict:
            # All years
            if ENTRY_TYPE_KEY in self.entrantDict and self.entrantDict[ENTRY_TYPE_KEY] == 'Youth Weekend':
                self.entrantDict[STATUS_KEY] = 'Youth Fleet'

            # Recent years
            elif self.entrantDict[STATUS_KEY] == 'Gold-Fleet':
                self.entrantDict[STATUS_KEY] = 'Gold Fleet'

            # 2010 + 2011
            elif self.entrantDict[STATUS_KEY] in ('Pro-Fleet', 'Professional'):
                self.entrantDict[STATUS_KEY] = 'Pro Fleet'

            # 2016 and maybe the future
            elif self.entrantDict[STATUS_KEY] in ('Youth-Fleet', 'Youth'):
                self.entrantDict[STATUS_KEY] = 'Youth Fleet'

            # All years
            elif self.entrantDict[STATUS_KEY] == 'Kite-Fleet':
                self.entrantDict[STATUS_KEY] = 'Kite Fleet'

            # All years
            elif self.entrantDict[STATUS_KEY] == 'Boat-Fleet':
                self.entrantDict[STATUS_KEY] = 'Boat Fleet'

            # All years
            elif self.entrantDict[STATUS_KEY] == 'Wing-Fleet':
                self.entrantDict[STATUS_KEY] = 'Wing Fleet'

            # Convert any unrecognised values to unknown
            if self.entrantDict[STATUS_KEY] not in config['Entrants'][STATUS_KEY]:
                self.entrantDict[STATUS_KEY] = 'TBC'
        else:
            self.entrantDict[STATUS_KEY] = 'TBC'


    def patchEntryType(self):
        '''Patch entry type in dictionary to handle previous years'''

        if ENTRY_TYPE_KEY in self.entrantDict:
            # 2010 + 2011 + 2012 + 2013
            if self.entrantDict[ENTRY_TYPE_KEY] in ('2 Days Only', '2 Days', '2-day'):
                self.entrantDict[ENTRY_TYPE_KEY] = '2-Day'

            # 2011 + 2012
            elif self.entrantDict[ENTRY_TYPE_KEY] in ('1 Day Only', '1 Day', '1-day'):
                self.entrantDict[ENTRY_TYPE_KEY] = '1-Day'

            # Convert any unrecognised values to unknown (e.g. 2010 + 2011 + 2012 saw empty values)
            if self.entrantDict[ENTRY_TYPE_KEY] not in config['Entrants'][ENTRY_TYPE_KEY]:
                self.entrantDict[ENTRY_TYPE_KEY] = 'TBC'
        else:
            self.entrantDict[ENTRY_TYPE_KEY] = 'TBC'


    def patchUKWA(self):
        '''Patch UKWA membership in dictionary to handle previous years'''

        if UKWA_MEMBER_KEY in self.entrantDict:
            if CRAFT_TYPE_KEY in self.entrantDict:
                if self.entrantDict[CRAFT_TYPE_KEY] == 'Sailboard':
                    self.entrantDict[UKWA_MEMBER_KEY] = self.entrantDict[UKWA_MEMBER_KEY][:1].upper()

                    if self.entrantDict[UKWA_MEMBER_KEY] not in config['Entrants'][UKWA_MEMBER_KEY]:
                        self.entrantDict[UKWA_MEMBER_KEY] = '?'
                elif self.entrantDict[CRAFT_TYPE_KEY] != 'TBC':
                    self.entrantDict[UKWA_MEMBER_KEY] = 'N'
                else:
                    self.entrantDict[UKWA_MEMBER_KEY] = '?'
            else:
                self.entrantDict[UKWA_MEMBER_KEY] = '?'
        else:
            self.entrantDict[UKWA_MEMBER_KEY] = '?'


    def patchISWC(self):
        '''Patch ISWC membership in dictionary to handle previous years'''

        if ISWC_MEMBER_KEY in self.entrantDict:
            if CRAFT_TYPE_KEY in self.entrantDict:
                if self.entrantDict[CRAFT_TYPE_KEY] == 'Sailboard':
                    self.entrantDict[ISWC_MEMBER_KEY] = self.entrantDict[ISWC_MEMBER_KEY][:1].upper()

                    if self.entrantDict[ISWC_MEMBER_KEY] not in config['Entrants'][ISWC_MEMBER_KEY]:
                        self.entrantDict[ISWC_MEMBER_KEY] = '?'
                elif self.entrantDict[CRAFT_TYPE_KEY] != 'TBC':
                    self.entrantDict[ISWC_MEMBER_KEY] = 'N'
                else:
                    self.entrantDict[ISWC_MEMBER_KEY] = '?'
            else:
                self.entrantDict[ISWC_MEMBER_KEY] = '?'
        else:
            self.entrantDict[ISWC_MEMBER_KEY] = '?'


    def validateDictionary(self):
        '''Validate the dictionary by comparing to known values in the application configuration'''

        for key in config['Entrants']:
            if key in self.entrantDict:
                if self.entrantDict[key] not in config['Entrants'][key]:
                    raise ValueError('Unexpected "{}" for entrant ID {} - "{}"'.format(
                        key, self.getValue(ID_KEY), self.entrantDict[key]))
            else:
                raise ValueError('Missing "{}" for entrant ID {}'.format(key, self.getValue(ID_KEY)))


    def parseGt31Details(self):
        '''Parse GT-31 details (user IDs and serial numbers) and record in sets'''

        if GT31_ID_KEY in self.entrantDict:
            self.gt31Ids = set()
            for gt31Id in str(self.entrantDict[GT31_ID_KEY].replace(';', ',')).split(','):
                if gt31Id:
                    self.gt31Ids.add(gt31Id)
        else:
            self.gt31Ids = None

        if GT31_SN_KEY in self.entrantDict:
            self.gt31SerialNumbers = set()
            for gt31SerialNumber in str(self.entrantDict[GT31_SN_KEY].replace(';', ',')).split(','):
                if gt31SerialNumber:
                    self.gt31SerialNumbers.add(gt31SerialNumber)
        else:
            self.gt31SerialNumbers = None


    def storeRun(self, courseDate, courseId, startTime, duration, speed, cog):
        '''Store run in memory'''

        #speedRun = SpeedRun(courseId, startTime, duration, speed, cog)

        if courseDate not in self.sessions:
            self.sessions[courseDate] = []

        # TODO
        # 1) Check youth results only apply to weekend
        # 2) Check people with 2 day entry have not exceeded 2 days
        #self.sessions[courseDate].append(speedRun)

## Unit Tests

Test the individual patch functions, etc.

In [5]:
class TestName(unittest.TestCase):
    '''Class to test name handling'''

    def testSimpleName(self):
        '''Test simple name variations'''

        entrant = Entrant(['First Name', 'Family Name'], ['Michael', 'George'])
        self.assertEqual(entrant.getValue(NAME_KEY), 'Michael George')


    def testPaddedName(self):
        '''Test whitespace removal'''

        entrant = Entrant(['First Name', 'Family Name'], [' Michael ', 'George'])
        self.assertEqual(entrant.getValue(NAME_KEY), 'Michael George')

        entrant = Entrant(['First Name', 'Family Name'], ['Michael', ' George '])
        self.assertEqual(entrant.getValue(NAME_KEY), 'Michael George')


    def testSingleName(self):
        '''Test single name (e.g. forename)'''

        entrant = Entrant(['First Name'], ['Michael'])
        self.assertEqual(entrant.getValue(NAME_KEY), 'Michael')

        entrant = Entrant(['Family Name'], ['George'])
        self.assertEqual(entrant.getValue(NAME_KEY), 'George')


    def testMissingName(self):
        '''Test missing name'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(NAME_KEY), 'TBC')


    def testEmptyName(self):
        '''Test empty name'''

        entrant = Entrant(['First Name', 'Family Name'], ['', ''])
        self.assertEqual(entrant.getValue(NAME_KEY), 'TBC')


    def testGpsName(self):
        '''Test GPS name'''

        entrant = Entrant(['GT31 ID'], ['GEORG8MICHA'])
        self.assertEqual(entrant.getValue(NAME_KEY), 'GEORG8MICHA')

In [6]:
class TestGender(unittest.TestCase):
    '''Class to test gender'''

    def testHeadings(self):
        '''Test title variations'''

        for heading in ['Gender', 'Male/Female']:
            for value in ['M', 'F']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(GENDER_KEY), value)


    def testMales(self):
        '''Test female variations'''

        for heading in ['Gender', 'Male/Female']:
            for value in ['M', 'm', 'Male', 'male']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(GENDER_KEY), 'M')


    def testFemales(self):
        '''Test female variations'''

        for heading in ['Gender', 'Male/Female']:
            for value in ['F', 'f', 'Female', 'female']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(GENDER_KEY), 'F')


    def testMissing(self):
        '''Test what happens when gender is missing'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(GENDER_KEY), '?')


    def testBlank(self):
        '''Test what happens when gender is blank'''

        for heading in ['Gender', 'Male/Female']:
            entrant = Entrant([heading], [''])
            self.assertEqual(entrant.getValue(GENDER_KEY), '?')


    def testInvalid(self):
        '''Test what happens when gender is invalid'''

        for heading in ['Gender', 'Male/Female']:
            for value in ['W', 'w', 'What', 'what']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(GENDER_KEY), '?')

In [7]:
class TestNationality(unittest.TestCase):
    '''Class to test nationality'''

    def testValidNationality(self):
        '''Test valid nationality - e.g. United Kingdom'''

        entrant = Entrant(['Country', 'Nationality'], ['England', 'United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Scotland', 'United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Wales', 'United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Northern Ireland', 'United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Sweden', 'Sweden'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'Sweden')

        entrant = Entrant(['Country', 'Nationality'], [' XXX ', ' XXX '])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'XXX')


    def testMissingNationality(self):
        '''Test missing nationality'''

        entrant = Entrant(['Country'], ['United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country'], ['England'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country'], ['Scotland'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country'], ['Wales'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country'], ['Northern Ireland'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country'], ['Sweden'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'Sweden')


    def testEmptyNationality(self):
        '''Test missing nationality'''

        entrant = Entrant(['Country', 'Nationality'], ['United Kingdom', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['England', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Scotland', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Wales', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Northern Ireland', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Sweden', ''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'Sweden')


    def testUnitedKingdom(self):
        '''Test valid nationality - e.g. United Kingdom'''

        entrant = Entrant(['Country', 'Nationality'], ['United Kingdom', 'United Kingdom'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['England', 'England'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Scotland', 'Scotland'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Wales', 'Wales'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')

        entrant = Entrant(['Country', 'Nationality'], ['Northern Ireland', 'Northern Ireland'])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'United Kingdom')


    def testMissing(self):
        '''Test unknown nationality'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'TBC')


    def testEmptyNationality(self):
        '''Test empty nationality'''

        entrant = Entrant(['Nationality'], [''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'TBC')


    def testEmptyCountry(self):
        '''Test empty country'''

        entrant = Entrant(['Country'], [''])
        self.assertEqual(entrant.getValue(NATIONALITY_KEY), 'TBC')

In [8]:
class TestCraftType(unittest.TestCase):
    '''Class to test craft type handling'''

    def testValid(self):
        '''Test valid craft types'''

        for craftType in ['Sailboard', 'Kiteboard', 'Wingboard', 'Boat']:
            entrant = Entrant(['Craft Type'], [craftType])
            self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), craftType)


    def testMissing(self):
        '''Test missing craft types'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'TBC')


    def testEmpty(self):
        '''Test empty craft types'''

        entrant = Entrant(['Craft Type'], [''])
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'TBC')


    def testInvalid(self):
        '''Test valid craft types'''

        for craftType in ['Powerboat', 'Jetski']:
            entrant = Entrant(['Craft Type'], [craftType])
            self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'TBC')

In [9]:
class TestNovice(unittest.TestCase):
    '''Class to test novice handling'''

    def testHeadings(self):
        '''Test title variations'''

        for heading in ['Novice(Y/N)']:
            for value in ['Y', 'N']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(NOVICE_KEY), value)


    def testYes(self):
        '''Test yes variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['Novice(Y/N)']:
                for value in ['Y', 'y', 'Yes', 'yes']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')


    def testNo(self):
        '''Test no variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['Novice(Y/N)']:
                for value in ['N', 'n', 'No', 'no']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')


    def testMissing(self):
        '''Test what happens when novice is missing'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(NOVICE_KEY), '?')

        for craftType in ['Sailboard', 'Kiteboard', 'Wingboard', 'Boat']:
            for status in ['Amateur', 'Youth-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, status])
                self.assertEqual(entrant.getValue(NOVICE_KEY), '?')

            for status in ['Gold-Fleet', 'Pro-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, status])
                self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')


    def testBlank(self):
        '''Test what happens when novice is blank'''

        for heading in ['Novice(Y/N)']:
            entrant = Entrant([heading], [''])
            self.assertEqual(entrant.getValue(NOVICE_KEY), '?')


    def testInvalid(self):
        '''Test what happens when novice is invalid'''

        for heading in ['Novice(Y/N)']:
            for value in ['W', 'w', 'What', 'what']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(NOVICE_KEY), '?')


    def testNovice(self):
        '''Test what happens when status is novice'''

        entrant = Entrant(['Status'], ['Novice'])
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')

        for heading in ['Novice(Y/N)']:
            for value in ['N', 'n', 'No', 'no']:
                entrant = Entrant(['Status', heading], ['Novice', value])
                self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')


    def testNonSailboard(self):
        '''Test what happens for non-windsurfers when novice is present'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['Novice(Y/N)']:
                for value in ['Y', 'y', 'Yes', 'yes']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')

                for value in ['N', 'n', 'No', 'no']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')

In [10]:
class TestFirstTimer(unittest.TestCase):
    '''Class to test first timer handling'''

    def testHeadings(self):
        '''Test title variations'''

        for heading in ['First Timer']:
            for value in ['Y', 'N']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), value)


    def testYes(self):
        '''Test yes variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['First Timer']:
                for value in ['Y', 'y', 'Yes', 'yes']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')


    def testNo(self):
        '''Test no variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['First Timer']:
                for value in ['N', 'n', 'No', 'no']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')


    def testMissing(self):
        '''Test what happens when first timer is missing'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), '?')

        for craftType in ['Sailboard', 'Kiteboard', 'Wingboard', 'Boat']:
            for status in ['Amateur', 'Youth-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, status])
                self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), '?')

            for status in ['Gold-Fleet', 'Pro-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, status])
                self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')


    def testBlank(self):
        '''Test what happens when first timer is blank'''

        for heading in ['First Timer']:
            entrant = Entrant([heading], [''])
            self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), '?')


    def testInvalid(self):
        '''Test what happens when first timer is invalid'''

        for heading in ['First Timer']:
            for value in ['W', 'w', 'What', 'what']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), '?')


    def testNonSailboard(self):
        '''Test what happens for non-windsurfers when first timer is present'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for heading in ['First Timer']:
                for value in ['Y', 'y', 'Yes', 'yes']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')

                for value in ['N', 'n', 'No', 'no']:
                    entrant = Entrant(['Craft Type', heading], [craftType, value])
                    self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')

In [11]:
class TestStatus(unittest.TestCase):
    '''Class to test status'''

    def testCraftTypes(self):
        '''Test sailboard variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for value in ['Youth']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, value])
                self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')

            for value in ['Amateur']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, value])
                self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')

            for value in ['Gold Fleet', 'Gold-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, value])
                self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')

            for value in ['Professional', 'Pro-Fleet']:
                entrant = Entrant(['Craft Type', 'Status'], [craftType, value])
                self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')

            for value in ['Novice', 'Amateur', 'Gold Fleet', 'Gold-Fleet', 'Professional', 'Pro Fleet', 'Pro-Fleet']:
                entrant = Entrant(['Craft Type', 'Status', 'Entry Type'], [craftType, value, 'Youth Weekend'])
                self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')


    def testInvalid(self):
        '''Test invalid status (e.g. "Please select" in 2012)'''

        # 2012 saw a status of "Please select"
        entrant = Entrant(['Craft Type', 'Status'], ['Sailboard', 'Please select'])
        self.assertEqual(entrant.getValue(STATUS_KEY), 'TBC')

In [12]:
class TestEntryType(unittest.TestCase):
    '''Class to test entry type'''

    def testHeadings(self):
        '''Test title variations'''

        for heading in ['Entry Type', 'EntryType', 'requiredEntryType']:
            for value in ['All Week', '2-Day', 'Youth Weekend']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), value)


    def testTwoDays(self):
        '''Test two day variations'''

        for heading in ['Entry Type', 'EntryType', 'requiredEntryType']:
            for value in ['2 Days Only', '2 Days', '2-day', '2-Day']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')


    def testOneDay(self):
        '''Test one day variations'''

        for heading in ['Entry Type', 'EntryType', 'requiredEntryType']:
            for value in ['1 Day Only', '1 Day', '1-day', '1-Day']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '1-Day')


    def testInvalid(self):
        '''Test invalid entry type (e.g. unspecified)'''

        for heading in ['Entry Type', 'EntryType', 'requiredEntryType']:
            for value in ['Eh?', '']:
                entrant = Entrant([heading], [value])
                self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'TBC')

In [13]:
class TestUKWA(unittest.TestCase):
    '''Class to test UKWA membership'''

    def testYes(self):
        '''Test yes variations'''

        for value in ['Y', 'y', 'Yes', 'yes']:
            entrant = Entrant(['Craft Type', 'UKWA Member'], ['Sailboard', value])
            self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')


    def testNo(self):
        '''Test no variations'''

        for value in ['N', 'n', 'No', 'no']:
            entrant = Entrant(['Craft Type', 'UKWA Member'], ['Sailboard', value])
            self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')


    def testMissing(self):
        '''Test missing UKWA membership'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')


    def testBlank(self):
        '''Test blank UKWA membership'''

        entrant = Entrant(['Craft Type', 'UKWA Member'], ['Sailboard', ''])
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')


    def testInvalid(self):
        '''Test invalid UKWA membership (e.g. unspecified)'''

        for value in ['W', 'w', 'What', 'what']:
            entrant = Entrant(['Craft Type', 'UKWA Member'], ['Sailboard', value])
            self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')


    def testNonWindsurfers(self):
        '''Test non-windsurfer variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for value in ['Y', 'y', 'Yes', 'yes']:
                entrant = Entrant(['Craft Type', 'UKWA Member'], [craftType, value])
                self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')

In [14]:
class TestISWC(unittest.TestCase):
    '''Class to test ISWC membership'''

    def testHeadings(self):
        '''Test title variations'''

        for heading in ['NMA Member', 'ISWC Member']:
            for value in ['Y', 'N']:
                entrant = Entrant(['Craft Type', heading], ['Sailboard', value])
                self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), value)


    def testYes(self):
        '''Test yes variations'''

        for value in ['Y', 'y', 'Yes', 'yes']:
            entrant = Entrant(['Craft Type', 'ISWC Member'], ['Sailboard', value])
            self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testNo(self):
        '''Test no variations'''

        for heading in ['NMA Member', 'ISWC Member']:
            for value in ['N', 'n', 'No', 'no']:
                entrant = Entrant(['Craft Type', heading], ['Sailboard', value])
                self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')


    def testMissing(self):
        '''Test missing ISWC membership'''

        entrant = Entrant()
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testBlank(self):
        '''Test blank ISWC membership'''

        for heading in ['NMA Member', 'ISWC Member']:
            entrant = Entrant(['Craft Type', heading], ['Sailboard', ''])
            self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testInvalid(self):
        '''Test invalid ISWC membership (e.g. unspecified)'''

        for heading in ['NMA Member', 'ISWC Member']:
            for value in ['W', 'w', 'What', 'what']:
                entrant = Entrant(['Craft Type', heading], ['Sailboard', value])
                self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testNonWindsurfers(self):
        '''Test non-windsurfer variations'''

        for craftType in ['Kiteboard', 'Wingboard', 'Boat']:
            for value in ['Y', 'y', 'Yes', 'yes']:
                entrant = Entrant(['Craft Type', 'ISWC Member'], [craftType, value])
                self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')

In [15]:
class TestGt31Serials(unittest.TestCase):
    '''Class to test GT-31 serial number'''

    def testSimple(self):
        '''Test single GPS'''

        entrant = Entrant(['GT31 SN'], ['932001290'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['932001290']))


    def testCommas(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 SN'], ['32001288,832001646,832004341'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))


    def testSemiColons(self):
        '''Test multiple GPS using semiocolons'''

        entrant = Entrant(['GT31 SN'], ['32001288;832001646;832004341'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))


    def testRedundantCommas(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 SN'], [',32001288,832001646,832004341,'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))

        entrant = Entrant(['GT31 SN'], [',,32001288,,832001646,,832004341,,'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))


    def testRedundantSemiColons(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 SN'], [';32001288;832001646;832004341;'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))

        entrant = Entrant(['GT31 SN'], [';;32001288;;832001646;;832004341;;'])
        self.assertEqual(entrant.gt31SerialNumbers, set(['32001288','832001646','832004341']))

In [16]:
class TestGt31Ids(unittest.TestCase):
    '''Class to test GT-31 usernames / IDs'''

    def testSimple(self):
        '''Test single GPS'''

        entrant = Entrant(['GT31 ID'], ['K47 ZARADAVIS'])
        self.assertEqual(entrant.gt31Ids, set(['K47 ZARADAVIS']))

        entrant = Entrant(['GT31 ID'], ['WSW 9'])
        self.assertEqual(entrant.gt31Ids, set(['WSW 9']))

        entrant = Entrant(['GT31 ID'], ['GEORG30MICHA'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA']))


    def testCommas(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 ID'], ['GEORG30MICHA,GEORG8MICHA'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))


    def testSemiColons(self):
        '''Test multiple GPS using semiocolons'''

        entrant = Entrant(['GT31 ID'], ['GEORG30MICHA;GEORG8MICHA'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))


    def testRedundantCommas(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 ID'], [',GEORG30MICHA,GEORG8MICHA,'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))

        entrant = Entrant(['GT31 ID'], [',,GEORG30MICHA,,GEORG8MICHA,,'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))


    def testRedundantSemiColons(self):
        '''Test multiple GPS using commas'''

        entrant = Entrant(['GT31 ID'], [';GEORG30MICHA;GEORG8MICHA;'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))

        entrant = Entrant(['GT31 ID'], [';;GEORG30MICHA;;GEORG8MICHA;;'])
        self.assertEqual(entrant.gt31Ids, set(['GEORG30MICHA','GEORG8MICHA']))

## Unit Tests

Test some examples from past years - 2010 onwards

In [17]:
class TestYear2010(unittest.TestCase):
    '''Class to test entrants from 2010'''

    def testYouthSailboard(self):
        '''Test using 2010 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Sailboard', 'Amateur', 'Y', 'Youth Weekend', 'N', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testFirstTimeSailboard(self):
        '''Test using 2010 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Sailboard', 'Amateur', 'Y', '', 'N', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Novice')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'TBC')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testAmateurSailboard(self):
        '''Test using 2010 amateur sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Sailboard', 'Amateur', 'N', 'All Week', '', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testGoldFleetSailboard(self):
        '''Test using 2010 gold fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Sailboard', 'Gold Fleet', 'N', 'All Week', '', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testProFleetSailboard(self):
        '''Test using 2010 pro fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Sailboard', 'Professional', 'N', 'All Week', '', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testKite(self):
        '''Test using 2010 kite data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['M', 'Kite Board', 'Professional', 'N', 'All Week', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Kiteboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')


    def testBoat(self):
        '''Test using 2010 boat data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'Novice(Y/N)', 'requiredEntryType', 'UKWA Member', 'NMA Member'],
            ['F', 'Boat', 'Amateur', 'N', '2 Days Only', 'N', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'F')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Boat')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')

In [18]:
class TestYear2011(unittest.TestCase):
    '''Class to test entrants from 2011'''

    def testYouthSailboard(self):
        '''Test using 2011 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Novice', 'Youth Weekend', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testFirstTimeSailboard(self):
        '''Test using 2011 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Novice', '', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Novice')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'Y')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'TBC')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testAmateurSailboard(self):
        '''Test using 2011 amateur sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['F', 'Sailboard', 'Amateur', '2 Days', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'F')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(NOVICE_KEY), '?')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testGoldFleetSailboard(self):
        '''Test using 2011 gold fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Gold Fleet', 'All Week', '', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testProFleetSailboard(self):
        '''Test using 2011 pro fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Professional', 'All Week', 'Y', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testKite(self):
        '''Test using 2011 kite data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['M', 'Kite Board', 'Professional', 'All Week', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Kiteboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')


    def testBoat(self):
        '''Test using 2011 boat data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'requiredEntryType', 'UKWA Member', 'ISWC Member'],
            ['F', 'Boat', 'Gold Fleet', '2 Days Only', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'F')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Boat')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')
        self.assertEqual(entrant.getValue(NOVICE_KEY), 'N')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')

In [19]:
class TestYear2012(unittest.TestCase):
    '''Class to test entrants from 2012'''

    def testYouthSailboard(self):
        '''Test using 2012 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Gold-Fleet', 'Youth Weekend', 'N', 'Yes', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testFirstTimeSailboard(self):
        '''Test using 2012 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Amateur', '', 'y', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'First Timer')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'TBC')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testAmateurSailboard(self):
        '''Test using 2012 amateur sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Amateur', '', 'N', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'TBC')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), '?')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testGoldFleetSailboard(self):
        '''Test using 2012 gold fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Gold-Fleet', 'All Week', 'N', 'No', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testProFleetSailboard(self):
        '''Test using 2012 pro fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Sailboard', 'Professional', 'All Week', 'N', 'Yes', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'Y')


    def testKite(self):
        '''Test using 2012 kite data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Kite Board', 'Amateur', 'All Week', 'N', 'No', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Kiteboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')


    def testBoat(self):
        '''Test using 2012 boat data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member', 'ISWC Member'],
            ['M', 'Boat', 'Boat-Fleet', '2-day', 'N', '', ''])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Boat')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Boat Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), 'N')

In [20]:
class TestYear2013(unittest.TestCase):
    '''Class to test entrants from 2013'''

    def testYouthSailboard(self):
        '''Test using 2013 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Amateur', 'Youth Weekend', 'Yes', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testFirstTimeSailboard(self):
        '''Test using 2013 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['F', 'Sailboard', 'Amateur', '2-Day', 'Yes', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'F')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'First Timer')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testAmateurSailboard(self):
        '''Test using 2013 amateur sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Amateur', 'All Week', 'No', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testGoldFleetSailboard(self):
        '''Test using 2013 gold fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Gold-Fleet', 'Youth Weekend', 'No', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testProFleetSailboard(self):
        '''Test using 2013 pro fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Professional', '2-day', 'No', 'Yes'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testKite(self):
        '''Test using 2013 kite data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Kite Board', 'Kite-Fleet', 'Youth Weekend', 'Yes', 'No'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Kiteboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testBoat(self):
        '''Test using 2013 boat data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Boat', 'Boat-Fleet', 'All Week', 'Yes', 'No'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Boat')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'First Timer')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')

In [21]:
class TestYear2016(unittest.TestCase):
    '''Class to test entrants from 2016'''

    def testYouthSailboard(self):
        '''Test using 2016 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Youth', 'Youth Weekend', 'N', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testFirstTimeSailboard(self):
        '''Test using 2016 first time sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Amateur', '2-Day', 'Y', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'First Timer')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testAmateurSailboard(self):
        '''Test using 2016 amateur sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Amateur', '2-Day', 'N', 'N'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Amateur')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testGoldFleetSailboard(self):
        '''Test using 2016 gold fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Gold-Fleet', 'All Week', 'N', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Gold Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testProFleetSailboard(self):
        '''Test using 2016 pro fleet sailboard data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Sailboard', 'Professional', '2-Day', 'N', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Sailboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Pro Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), '2-Day')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'Y')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testKite(self):
        '''Test using 2016 kite data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Kite Board', 'Youth', 'Youth Weekend', 'N', 'Y'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Kiteboard')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'Youth Fleet')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'Youth Weekend')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'N')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')


    def testBoat(self):
        '''Test using 2016 boat data'''

        entrant = Entrant(
            ['Male/Female', 'Craft Type', 'Status', 'EntryType', 'First Timer', 'UKWA Member'],
            ['M', 'Boat', 'Boat-Fleet', 'All Week', 'Y', 'N'])

        self.assertEqual(entrant.getValue(GENDER_KEY), 'M')
        self.assertEqual(entrant.getValue(CRAFT_TYPE_KEY), 'Boat')
        self.assertEqual(entrant.getValue(STATUS_KEY), 'First Timer')
        self.assertEqual(entrant.getValue(ENTRY_TYPE_KEY), 'All Week')
        self.assertEqual(entrant.getValue(FIRST_TIMER_KEY), 'Y')
        self.assertEqual(entrant.getValue(UKWA_MEMBER_KEY), 'N')
        self.assertEqual(entrant.getValue(ISWC_MEMBER_KEY), '?')

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [22]:
# Read main config into global variable
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....................................................................................................
----------------------------------------------------------------------
Ran 101 tests in 0.275s

OK


## All Done!